<p style="padding: 10px; background-color:#FFA500;color: Black;font-weight: bold;
              text-align: center; font-size:220%;border-radius:10px 80px;">Проверка работоспособности api</p>


Перед началом необходимо собрать docker-образ и запустить контейнер

In [1]:
import pandas as pd
import urllib.request
import json 
from sklearn.metrics import roc_auc_score,roc_curve
from urllib import request, parse

def get_prediction(field_array=None):
    """
    Input parameters:
        field_array: python list - contains raw fields needed to make a prediction
        myurl: REST API url
    Returns:
        result: float - model prediction
    """

    age, sex, cp, trestbps, chol, fbs, restecg, thalach, exang, oldpeak, slope, ca, thal = field_array
    
    body = {'age': age, 
            'sex': sex,
            'cp': cp,
            'trestbps': trestbps, 
            'chol': chol,
            'fbs': fbs,
            'restecg': restecg,
            'thalach': thalach,
            'exang': exang,
            'oldpeak': oldpeak, 
            'slope':slope, 
            'ca': ca,
            'thal':thal
           } 

    myurl = "http://192.168.1.224:8180/predict"
    req = urllib.request.Request(myurl)
    req.add_header('Content-Type', 'application/json; charset=utf-8')
    jsondata = json.dumps(body)
    jsondataasbytes = jsondata.encode('utf-8')   # needs to be bytes
    req.add_header('Content-Length', len(jsondataasbytes))
    response = urllib.request.urlopen(req, jsondataasbytes)
    
    return json.loads(response.read())['predictions']

X_test = pd.read_csv("X_test.csv")
y_test = pd.read_csv("y_test.csv")

Посмотрим что за данные у нас есть

In [2]:
X_test.head(3)

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal
0,58,1,0,150,270,0,0,111,1,0.8,2,0,3
1,60,1,0,130,253,0,1,144,1,1.4,2,1,3
2,45,0,1,112,160,0,1,138,0,0.0,1,0,2


### Getting predictions from REST API

In [3]:
%%time
predictions = X_test.iloc[:500].apply(lambda x: get_prediction(x), 1)

CPU times: user 78.2 ms, sys: 17.1 ms, total: 95.3 ms
Wall time: 5.13 s


check performance (roc auc)

In [4]:
roc_auc_score(y_score=predictions.values, y_true=y_test.iloc[:500])

0.90801393728223